In [ ]:
import os
import time
import input_data
import tensorflow as tf
import numpy as np

class SPPLayer():
    def __init__(self,bins,feature_map_size):
        self.strides = []
        self.filters = []
#        print(type(feature_map_size))
        self.a = float(feature_map_size)
        self.bins = bins
        self.n = len(bins)

    def spatial_pyramid_pooling(self,data):
        self.input = data
        self.map_nums=self.input.get_shape().as_list()[3]
        for i in range(self.n):
            x = int(math.floor(self.a/float(self.bins[i])))
            self.strides.append(x)
            x = int (math.ceil(self.a/float(self.bins[i])))
            self.filters.append(x)  
        print('bin_strides', self.strides)
        print('bin_filters', self.filters)
        
        self.pooled_out = []
        for i in range(self.n):
            self.pooled_out.append(tf.nn.max_pool(self.input,
                ksize=[1, self.filters[i], self.filters[i], 1], 
                strides=[1, self.strides[i], self.strides[i], 1],
                padding='VALID'))
        for i in range(self.n):
            self.pooled_out[i] = tf.reshape(self.pooled_out[i], [-1, self.bins[i]*self.bins[i]*self.map_nums])
            
        print('pooled_out', self.pooled_out)
        self.output = tf.concat( [self.pooled_out[0], self.pooled_out[1], self.pooled_out[2]],1)
        print(self.output.shape)
        return self.output
    
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)  
sess = tf.InteractiveSession()
img = tf.placeholder("float", shape=[None, 784])
train_data = tf.reshape(img, [-1,28,28,1])
label = tf.placeholder('float', shape=[None, 10])
keep_prob = tf.placeholder("float")


with tf.variable_scope('conv1'):
    w1 = tf.get_variable(name='w1', shape=[5,5,1,10])
    b1 = tf.get_variable(name='b1', shape=[10])
conv1 =tf.nn.relu( tf.nn.conv2d(train_data, w1, strides=[1,1,1,1], padding='SAME') + b1 )
pool1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
print ('pool1.shape' , pool1.shape)

with tf.variable_scope('conv2'):
    w2 = tf.get_variable(name='w2', shape=[5,5,10,10])
    b2 = tf.get_variable(name='b2', shape=[10])
conv2 =tf.nn.relu( tf.nn.conv2d(pool1, w2, strides=[1,1,1,1], padding='SAME') + b2 )
pool2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
print ('pool2.shape' , pool2.shape)
fc = tf.reshape(pool2, [-1,7*7*10])
print ('fc.shape' , fc.shape)

with tf.variable_scope('fc1'):
    w3 = tf.get_variable(name='w3', shape=[7*7*10, 20])
    b3 = tf.get_variable(name='b3', shape=[20])
fc1 = tf.nn.relu(tf.matmul(fc, w3) + b3)
print ('fc1.shape' , fc1.shape)

with tf.variable_scope('fc2'):
    w4 = tf.get_variable(name='w4', shape=[20, 10])
    b4 = tf.get_variable(name='b4', shape=[10])
conv_result = tf.nn.softmax(tf.matmul(fc1, w4) + b4)
print ('conv_result.shape' , conv_result.shape)

loss = - tf.reduce_sum(label * tf.log(conv_result))
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
correct_prediction = tf.equal(tf.argmax(conv_result,1), tf.argmax(label,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.global_variables_initializer())

for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        print(batch[0].shape ,batch[1].shape)
        train_accuracy = accuracy.eval(feed_dict={img:batch[0], label: batch[1], keep_prob: 1.0})
        print ("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={img: batch[0], label: batch[1], keep_prob: 0.5})

print ("test accuracy %g"%accuracy.eval(feed_dict={img: mnist.test.images, label: mnist.test.labels, keep_prob: 1.0}))

In [25]:
import os
import time
import input_data
import tensorflow as tf
import numpy as np
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [28]:
train_data = mnist.train.images
train_label = mnist.train.labels
test_data = mnist.test.images
test_label = mnist.test.labels
np.save("mnist_train_data.npy",train_data)
np.save("mnist_train_label.npy",train_label)
np.save("mnist_test_data.npy",test_data)
np.save("mnist_test_label.npy",test_label)

In [16]:
import numpy as np
class data_label_data():
    def __init__(self, data, label):
        self.data = data
        self.label = label
    def next_batch(self, batch_size):
        label_len = len(self.label)
        shuffle_index = np.arange(label_len)
        np.random.shuffle(shuffle_index)
        print (shuffle_index)
        batch_index = shuffle_index[:batch_size]
        data_batch = self.data[batch_index]
        label_batch = self.label[batch_index]
        return [data_batch, label_batch]
class load_data(data_label_data):
    def __init__(self, train, test):
        self.train = train
        self.test = test
train_part = data_label_data(train_data, train_label)
test_part = data_label_data(test_data, train_label)
my_mnist = load_data(train_part, test_part)

In [23]:
my_mnist.train.next_batch(1)[0][0]

[ 6606 32798  5383 ..., 40647 32781  2444]


array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  